In [1]:
import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F

In [2]:
fc = nn.Linear(in_features=4, out_features=3, bias=False)

In [3]:
torch.Size([16])
    # 1d Tensor : [batch_size]
    # used for target labels or predictions.
torch.Size([16, 256])
    # 2D- Tensor : [batch_size, num_features (aka: C * H * W)]
    # use for nn.Linear() input.
torch.Size([10, 1, 2048])
    # 3-D Tensor : [batch_size, channels, num_features (aka: H * W)]
    # when used as nn.Conv1d() input.
    # (but [seq_len, batch_size, num_features]
    # if feeding an RNN).
torch.Size([16, 3, 28, 28])
    # 4-D Tensor : [batch_size, channels, height, width]
    # use for nn.Conv2d() input.
torch.Size([32, 1, 5, 15, 15])
    # 5D-Tensor: [batch_size, channels, depth, height, width]
    # use for nn.Conv3d() input.

torch.Size([32, 1, 5, 15, 15])

## I want to pass in a 28 x 28 image into a linear layer

Lets say my `image` vector is of size 28 * 28

But this 28 x 28 pixel image can’t be input as a [28, 28] tensor and nn.Linear() will accept only following form of Tensor

#### [batch_size, num_features]

Hence, in nn.Linear() layer the [28, 28] tensor will be interpreted as 28 batches and 28 num_features.

So to correct this, I need to reshape the input vector of [28, 28] to the form of [batch_size, num_features]

In [4]:
batch_size = 1

input_image = torch.randn(1, 28, 28)

print(input_image.size())

input_image = input_image.view(batch_size, -1)

print(input_image.size())

torch.Size([1, 28, 28])
torch.Size([1, 784])


In [5]:
fc = torch.nn.Linear(784, 10)

# Pass in the simulated image to the layer.

output = fc(input_image)
print(output.shape)


torch.Size([1, 10])


## Example from PyTorch [Official Documentation](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html)

In [22]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension. And 16*5*5 gives me 400
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) 
        # print('4-D Tensor Shape of x BEFORE Linear layer & before reshaping ', x.shape)  
        
        # x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = x.view(x.shape[0], -1)
        
        # print('Shape of x AFTER re-shaping before feeding to first Linear layer ', x.shape)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

input = torch.randn(32, 1, 32, 32)

output = net(input)
output.shape

torch.Size([32, 10])

### Another example of a Random Neural Network

In [25]:
class CNNClassifier(torch.nn.Module):
    def __init__(self, C=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.maxpool = nn.MaxPool2d(kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3)
        
        self.linear1 = nn.Linear(32 * 7 * 7, C) #1568
        self.softmax1 = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(F.leaky_relu(x))
        x = self.conv2(x)
        x = self.maxpool(F.leaky_relu(x))
        print('Shape of x BEFORE Linear layer ', x.shape)
        
        fc1 = self.linear1(x.view(x.shape[0], -1))
        
        print('Shape of x AFTER the first Linear layer ', fc1.shape)
        
        final_layer = self.softmax1(fc1)
        return final_layer
    
net = CNNClassifier()
input = torch.randn(32, 3, 64, 64)
out = net(input)

Shape of x BEFORE Linear layer  torch.Size([32, 32, 7, 7])
Shape of x AFTER the first Linear layer  torch.Size([32, 10])


### Summarize - The rule

## For transitioning from a convolutional layer output to a linear layer input - I must resize Conv Layer output which is a 4d-Tensor to a 2d-Tensor using view.

#### An example, a conv output of [32, 21, 50, 50] should be “flattened” to become a [32, 21 * 50 * 50] tensor. 

#### And the in_features of the linear layer should also be set to [21 * 50 * 50].